In [ ]:
!nvidia-smi

Mon Oct 30 09:26:26 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla T4            Off  | 00000000:00:05.0 Off |                    0 |
| N/A   

In [ ]:
!pip install  dgl -f https://data.dgl.ai/wheels/cu117/repo.html
!pip install  dglgo -f https://data.dgl.ai/wheels-test/repo.html

Looking in links: https://data.dgl.ai/wheels/cu117/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.7/91.7 MB 4.9 MB/s eta 0:00:0000:01m00:01
Looking in links: https://data.dgl.ai/wheels-test/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 46.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 77.4 MB/s eta 0:00:00:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.0/120.0 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.5/83.5 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.7/92.7 kB 8.7 MB/s eta 0:00:00
     ━━━━━

In [ ]:
import torch
import dgl
import yaml
import pandas as pd
import numpy as np
import gensim
from gensim.models import word2vec
import scipy.sparse as sp
from dgl.nn import SAGEConv
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
import itertools
import os

DGL backend not selected or invalid.  Assuming PyTorch for now.
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


In [ ]:
torch.__version__

'2.0.0'

# Loading the Data

In [ ]:
data = pd.read_csv("/kaggle/input/pmbok-csv/PMBOK.csv", index_col=0)

In [ ]:
data.head()

,title,text,clean_text,keywords,triplets,summary,definition
0,PROJECT RISK MANAGEMENT\n,Project Risk Management includes the processes...,project risk management include the process of...,"[(78.19565217391303, 'agreed - upon risk respo...","[{'head': 'monitoring risk', 'type': 'subclass...",[{'summary_text': ' project risk management in...,to increase the probability and/or impact of p...
1,KEY CONCEPTS FOR PROJECT RISK MANAGEMENT\n,All projects are risky since they are unique u...,all project be risky since they be unique unde...,"[(51.333333333333336, 'mitigate negative risk ...","[{'head': 'project risk', 'type': 'subclass of...",[{'summary_text': 'project risk management aim...,measurable risk threshold
2,TRENDS AND EMERGING PRACTICES IN PROJECT RISK ...,The focus of project risk management is broade...,the focus of project risk management be broade...,"[(63.667080745341615, 'coordinated approach to...","[{'head': 'non-event risk', 'type': 'subclass ...",[{'summary_text': 'Non-event risk need to be i...,non-event risk
3,TAILORING CONSIDERATIONS\n,"Because each project is unique, it is necessar...",because each project be unique it be necessary...,"[(33.5, 'produce breakthrough opportunity addr...","[{'head': 'project risk management', 'type': '...",[{'summary_text': 'considerations for tailor i...,project size
4,CONSIDERATIONS FOR AGILE/ADAPTIVE ENVIRONMENTS\n,"High-variability environments, by definition, ...",high-variability environment by definition inc...,"[(59.66666666666667, 'cross - functional proje...","[{'head': 'uncertainty', 'type': 'subclass of'...",[{'summary_text': ' high-variability environme...,high-variability environment


In [ ]:
data.fillna("", inplace=True)

# Create embeddings for entities and relationships

In [ ]:
# Building entities and relationships Corpus
corpus = []
for triplet_list in data.triplets.values:
    if triplet_list !="":
        parsed_triplet_list = yaml.safe_load(triplet_list)
        for triplet in parsed_triplet_list:
            corpus.append(list(triplet.values())[:3])

In [ ]:
len(corpus)

1082

In [ ]:
corpus[0:5]

[['monitoring risk', 'subclass of', 'risk management'],
 ['monitoring risk', 'facet of', 'risk management'],
 ['monitoring risk', 'part of', 'risk management'],
 ['negativerisk', 'subclass of', 'risks'],
 ['negativerisks', 'subclass of', 'risks']]

In [ ]:
# Intialize the word2Vec model using the Skip-Gram model (sg=1)
word2vec_model = word2vec.Word2Vec(corpus, vector_size=100, window=20, min_count=1, workers=4, sg=1)

In [ ]:
# calculating the word-vector of the a random entity/relationship.
word2vec_model.wv['risk']

array([-9.59447399e-03,  1.07108690e-02,  4.27725696e-04, -1.48455321e-03,
        5.49839018e-03, -7.20037660e-03,  4.76040225e-03,  1.08324178e-02,
        3.20952362e-03, -8.90230294e-03,  9.65807680e-03,  2.72254623e-03,
        2.33180774e-03, -5.24623692e-03,  8.13090242e-03, -3.00055626e-03,
        1.04825450e-02, -6.34215027e-03, -9.68244392e-03,  1.38124614e-03,
        2.85710511e-03, -1.87383278e-03,  1.34428255e-02,  8.68362468e-03,
       -9.50945914e-03,  2.74429959e-03,  4.42167744e-03,  2.77778576e-03,
       -2.84511363e-04,  1.61899929e-03,  3.25911725e-03, -2.99195712e-03,
       -5.93683682e-03, -4.98316996e-03,  2.74868379e-03,  1.18256807e-02,
        9.63415205e-03, -7.16643361e-03, -9.45503637e-03,  6.20032288e-03,
        4.25171060e-03,  2.90821469e-03,  5.14972955e-03, -3.06639797e-03,
        9.33401100e-03,  2.91885273e-03,  1.72931340e-03, -2.78675556e-03,
       -1.26651395e-03, -1.86577835e-03,  4.64809733e-03, -1.95868267e-03,
       -9.07983445e-03, -

In [ ]:
# Building embeddings dictionary : key: entity/relation Value: embedding
embeddings = {}
for document in corpus:
    for token in document:
        if token in word2vec_model.wv:
            embeddings[token] = word2vec_model.wv[token]

In [ ]:
len(embeddings)

649

# Building the entities relationships Graph

1.   Élément de liste
2.   Élément de liste



In [ ]:
def build_nodes_index(nodes_list):
    unique_nodes_list = []
    for node in nodes_list:
        if node not in unique_nodes_list:
            unique_nodes_list.append(node)

    nodes_index = {}
    index = 0
    for node in unique_nodes_list:
        nodes_index[node] = index
        index+=1
    return nodes_index

In [ ]:
def generate_nodes_indices(nodes_list, nodes_index):
    nodes_indices = []
    for node in nodes_list:
        nodes_indices.append(nodes_index[node])
    return nodes_indices

In [ ]:
source_nodes_concepts = []
destination_nodes_concepts = []
edges_relations = []

for document in corpus:
    source_nodes_concepts.append(document[0])
    destination_nodes_concepts.append(document[1])
    edges_relations.append(document[2])

In [ ]:
nodes = source_nodes_concepts + destination_nodes_concepts

nodes_index = build_nodes_index(nodes)

In [ ]:
source_nodes = generate_nodes_indices(source_nodes_concepts, nodes_index)
destination_nodes = generate_nodes_indices(destination_nodes_concepts, nodes_index)

In [ ]:
nodes_concepts_embeddings = [word2vec_model.wv[concept] for concept in nodes_index.keys()]

In [ ]:
edges_relations_embeddings = [word2vec_model.wv[edge_relation] for edge_relation in edges_relations]

In [ ]:
np.argmax(source_nodes)

1078

In [ ]:
source_nodes[-1]

156

In [ ]:
# Building the graph
graph = dgl.graph((source_nodes, destination_nodes))

In [ ]:
# print number of nodes
print("Number of nodes: {}\n".format(graph.num_nodes()))
# print node IDs
print("Node IDs are: \n{}\n".format(graph.nodes()))

# print number of edges
print("Number of edges: {}\n".format(graph.num_edges()))
# print the source and destination nodes of every edge
print("Source & destination nodes of every edge are: \n{}\n".format(graph.edges()))

# print edge IDs
print("Edge IDs are: \n{}".format(graph.edge_ids(source_nodes, destination_nodes)))

Number of nodes: 465

Node IDs are: 
tensor([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
         28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
         42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
         56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
         70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
         84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
         98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111,
        112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
        126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139,
        140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153,
        154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167,
        168, 169, 170, 171,

# Manipulating Node and Edge Features

In [ ]:
# aadd_newdoc_ufuncssign a 100-dimensional node feature vector called concept for each node
graph.ndata['concept'] = torch.from_numpy(np.array(nodes_concepts_embeddings))

In [ ]:
# assign a 100-dimensional edge feature vector called relation for each edge
graph.edata['relation'] = torch.from_numpy(np.array(edges_relations_embeddings))

In [ ]:
# print ndata
print("Node data: \n{}\n".format(graph.ndata))

# print edata
print("Edge data: \n{}".format(graph.edata))

Node data: 
{'concept': tensor([[-0.0034,  0.0020,  0.0078,  ...,  0.0041, -0.0044, -0.0001],
        [ 0.0015,  0.0042,  0.0009,  ..., -0.0014,  0.0074,  0.0091],
        [-0.0015, -0.0013, -0.0027,  ...,  0.0066, -0.0040, -0.0005],
        ...,
        [ 0.0096, -0.0054,  0.0093,  ..., -0.0065, -0.0090, -0.0088],
        [ 0.0077, -0.0061,  0.0031,  ...,  0.0018,  0.0034, -0.0032],
        [ 0.0087,  0.0031, -0.0006,  ..., -0.0090,  0.0085,  0.0090]])}

Edge data: 
{'relation': tensor([[ 0.0071, -0.0034,  0.0097,  ..., -0.0061,  0.0050, -0.0047],
        [ 0.0071, -0.0034,  0.0097,  ..., -0.0061,  0.0050, -0.0047],
        [ 0.0071, -0.0034,  0.0097,  ..., -0.0061,  0.0050, -0.0047],
        ...,
        [-0.0081,  0.0052,  0.0027,  ...,  0.0080,  0.0081,  0.0065],
        [-0.0081,  0.0052,  0.0027,  ...,  0.0080,  0.0081,  0.0065],
        [-0.0081,  0.0052,  0.0027,  ...,  0.0080,  0.0081,  0.0065]])}


Below we will use the [GraphConv](https://docs.dgl.ai/en/0.9.x/generated/dgl.nn.pytorch.conv.GraphConv.html) module from DGL to implement our GNN architecture. By stacking `N` GCN layers, the feature representations are updated with information of nodes up to `N` hops away. This is often treated as a hyperparameter for model tuning.

# Link Prediction using GNN

### Split data to training and testing sets

In [ ]:
# Split edge set for training and testing
source_nodes, destination_nodes = graph.edges()

edge_ids = np.arange(graph.num_edges())
edge_ids = np.random.permutation(edge_ids)

test_size = int(len(edge_ids) * 0.1)
train_size = graph.num_edges() - test_size

test_pos_source_nodes, test_pos_destination_nodes = source_nodes[edge_ids[:test_size]], destination_nodes[edge_ids[:test_size]]
train_pos_source_nodes, train_pos_destination_nodes = source_nodes[edge_ids[test_size:]], destination_nodes[edge_ids[test_size:]]

# Find all negative edges and split them for training and testing
adj = sp.coo_matrix((np.ones(len(source_nodes)), (source_nodes.numpy(), destination_nodes.numpy())), shape=(graph.num_nodes(), graph.num_nodes()))
adj_neg = 1 - adj.todense() - np.eye(graph.num_nodes())
neg_source_nodes, neg_destination_nodes = np.where(adj_neg != 0)

neg_edge_ids = np.random.choice(len(neg_source_nodes), graph.num_edges())

test_neg_source_nodes, test_neg_destination_nodes = neg_source_nodes[neg_edge_ids[:test_size]], neg_destination_nodes[neg_edge_ids[:test_size]]
train_neg_source_nodes, train_neg_destination_nodes = neg_source_nodes[neg_edge_ids[test_size:]], neg_destination_nodes[neg_edge_ids[test_size:]]

In [ ]:
train_graph = dgl.remove_edges(graph, edge_ids[:test_size])

In [ ]:
train_pos_graph = dgl.graph((train_pos_source_nodes, train_pos_destination_nodes), num_nodes=graph.num_nodes())
train_neg_graph = dgl.graph((train_neg_source_nodes, train_neg_destination_nodes), num_nodes=graph.num_nodes())

test_pos_graph = dgl.graph((test_pos_source_nodes, test_pos_destination_nodes), num_nodes=graph.num_nodes())
test_neg_graph = dgl.graph((test_neg_source_nodes, test_neg_destination_nodes), num_nodes=graph.num_nodes())

In [ ]:
# ----------- 2. create model -------------- #
# build a two-layer GraphSAGE model
class GraphSAGE(nn.Module):
    def __init__(self, in_feats, h_feats):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_feats, h_feats, "mean")
        self.conv2 = SAGEConv(h_feats, h_feats, "mean")

    def forward(self, graph, in_feat):
        h = self.conv1(graph, in_feat)
        h = F.relu(h)
        h = self.conv2(graph, h)
        return h

In [ ]:
class DotPredictor(nn.Module):
    def forward(self, graph, h):
        with graph.local_scope():
            graph.ndata["h"] = h
            # Compute a new edge feature named 'score' by a dot-product between the
            # source node feature 'h' and destination node feature 'h'.
            graph.apply_edges(fn.u_dot_v("h", "h", "score"))
            # u_dot_v returns a 1-element vector for each edge so you need to squeeze it.
            return graph.edata["score"][:, 0]

In [ ]:
model = GraphSAGE(train_graph.ndata["concept"].shape[1], 16)
# You can replace DotPredictor with MLPPredictor.
# pred = MLPPredictor(16)
predictor = DotPredictor()

In [ ]:
def compute_loss(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score])
    labels = torch.cat(
        [torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])]
    )
    return F.binary_cross_entropy_with_logits(scores, labels)


def compute_auc(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score]).numpy()
    labels = torch.cat(
        [torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])]
    ).numpy()
    return roc_auc_score(labels, scores)

In [ ]:
# ----------- 3. set up loss and optimizer -------------- #
# in this case, loss will in training loop
optimizer = torch.optim.Adam(
    itertools.chain(model.parameters(), predictor.parameters()), lr=0.01
)

In [ ]:
# ----------- 4. training -------------------------------- #
all_logits = []
epochs = 100
for e in range(epochs):
    # forward
    h = model(train_graph, train_graph.ndata["concept"])
    pos_score = predictor(train_pos_graph, h)
    neg_score = predictor(train_neg_graph, h)
    loss = compute_loss(pos_score, neg_score)

    # backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if e % 5 == 0:
        print("In epoch {}, loss: {}".format(e, loss))

In epoch 0, loss: 0.7108930349349976
In epoch 5, loss: 0.6588453054428101
In epoch 10, loss: 0.6230385303497314
In epoch 15, loss: 0.5798932909965515
In epoch 20, loss: 0.5443207621574402
In epoch 25, loss: 0.521355152130127
In epoch 30, loss: 0.5116016864776611
In epoch 35, loss: 0.5095983743667603
In epoch 40, loss: 0.5072599649429321
In epoch 45, loss: 0.5026677846908569
In epoch 50, loss: 0.4983031749725342
In epoch 55, loss: 0.4953477084636688
In epoch 60, loss: 0.49304571747779846
In epoch 65, loss: 0.4907679557800293
In epoch 70, loss: 0.4885837733745575
In epoch 75, loss: 0.48671936988830566
In epoch 80, loss: 0.4855210483074188
In epoch 85, loss: 0.4845995306968689
In epoch 90, loss: 0.4832878112792969
In epoch 95, loss: 0.4825465679168701


In [ ]:
# ----------- 5. check results ------------------------ #
from sklearn.metrics import roc_auc_score

with torch.no_grad():
    pos_score = predictor(test_pos_graph, h)
    neg_score = predictor(test_neg_graph, h)
    print("AUC", compute_auc(pos_score, neg_score))

AUC 0.9329561042524005


In [ ]:
unseen_graph = dgl.graph((test_pos_source_nodes, test_pos_destination_nodes), num_nodes=graph.num_nodes())

In [ ]:
def
unseen_source_nodes = []
unseen_destination_nodes = []
unseen_graph = dgl.graph((unseen_source_nodes, unseen_destination_nodes))

In [ ]:
graph.ndata['concept'][0]

In [ ]:
pos_score = predictor(unseen_graph, )
neg_score = predictor(unseen_graph, h)